Transform our posts collection to prepare it for the model: 
1- unset the shortcode field 
2- group posts by owner id
3- concatenate captions
4- export data to a new collection

In [ ]:
from pymongo import MongoClient

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['InfluencersMarketing']['newPosts'].aggregate([
    {
        '$unset': 'shortcode'
    }, {
        '$group': {
            '_id': '$owner.id', 
            'posts': {
                '$push': '$$ROOT.caption'
            }
        }
    }, {
        '$project': {
            'posts': {
                '$reduce': {
                    'input': '$posts', 
                    'initialValue': '', 
                    'in': {
                        '$concat': [
                            '$$value', '$$this'
                        ]
                    }
                }
            }
        }
    }, {
        '$out': {
            'db': 'InfluencersMarketing', 
            'coll': 'Posts'
        }
    }
])